### Team - RoundRobinGroupChat

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console

import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("deployment_name"),
    model=os.getenv("model_name"),
    api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("azure_endpoint"),
    api_key=os.getenv("api_key")
)

In [ ]:
blog_post_draft_agent = AssistantAgent(
    "BlogPostDraftAgent",
    model_client=model_client,
    system_message="You are a blog post draft agent. Your task is to create a draft for a blog post based on the provided topic.",
)

blog_post_enhancement_agent = AssistantAgent(
    "BlogPostEnhancementAgent",
    model_client=model_client,
    system_message="""You are a blog post enhancement agent. 
            Your task is to enhance the draft provided by the drafting agent.
            Focus on improving readability, adding relevant examples, and optimizing for SEO.
            If you're satisfied with the final draft, respond with 'FINAL'.
        """,
)

text_termination = TextMentionTermination("FINAL")

team = RoundRobinGroupChat(
    [blog_post_draft_agent, blog_post_enhancement_agent],
    termination_condition=text_termination,
)

In [ ]:
await Console(
    team.run_stream(
        task="Create a blog post about the benefits of meditation.")
)